In [40]:
#API
from cgi import test
import json
from typing import Optional
import pandas as pd
import requests
from http.client import HTTPSConnection
from base64 import b64encode
import csv
from urllib.request import Request, urlopen
import time
from collections import defaultdict

# auth= '464c01f7e53b2a5e6f00d9a1'

def csv_files(file):  
    with open(file, encoding="utf-8") as csv:
        file = pd.read_csv(csv)
    print("CSV INFO")
    print(file.head())
    return file

testCSV = csv_files("/Users/chaualala/Desktop/UZH/MSc Geographie/2. Semester/GEO885 - GIS Science Project/GEO885/API/Test_Data.csv")
print(testCSV)

with open("/Users/chaualala/Desktop/UZH/MSc Geographie/2. Semester/GEO885 - GIS Science Project/GEO885/API/Test_Data.csv", newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        print(reader)
        for row in reader:
            print(row['DEPARTURE_AIRPORT'], row['ARRIVAL_AIRPORT'], row["cabin_class"], row["currencies"])
print(row)
print(type(row))


CSV INFO
     Unnamed: 0   RecID     year Flugnummer cabin_class  DEPARTURE_AIRPORT  \
1 1        2020  LX4425  economy        CGN         ZRH                  1   
2 2        2020   OS420  economy        CDG         VIE                  1   
3 3        2020   OS553  economy        VIE         ZRH                  1   
4 4        2020   OS763  economy        VIE         VAR                  1   
5 5        2020   OS764  economy        VAR         VIE                  1   

     ARRIVAL_AIRPORT Anzahl Flugsegmente  distance currencies  \
1 1              441                 USD        70         DE   
2 2             1135                 USD       150         FR   
3 3              704                 USD       100         AT   
4 4             1127                 USD       220         AT   
5 5             1127                 USD       210         BG   

    EMISSIONS_KGCO2EQ      dep_country dep_coordinates arr_country  \
1 1    c(7.1427397728   50.8658981323)              CH  c(8.5

In [41]:
print("DATA")
def dic(row):
    gaz = {}
    origin = row['DEPARTURE_AIRPORT']
    destination = row['ARRIVAL_AIRPORT']
    cabin_class = row['cabin_class']
    currencies = row["currencies"]
    gaz.update({"segments": [{ "origin": origin, "destination": destination},], "cabin_class": cabin_class, "currencies":[currencies]})
    return gaz
data = dic(row)
print(data)

# print("Payload ")
# def emissions(row):
#     result = defaultdict(list)
#     for org, arr, abc, cur in zip(*row.values()):
#         result["segments"].append({
#                 "origin": org,
#                 "destination": arr})
#         result["cabin_class"].append(abc)
#         result["currencies"].append(cur)
#     return dict(result)
# print(emissions(row))


DATA
{'segments': [{'origin': 'ZRH', 'destination': 'ATL'}], 'cabin_class': 'economy', 'currencies': ['USD']}


In [42]:
#data = {"segments" : [{"origin" : "CGN","destination" : "ZRH"},],"cabin_class" : "economy","currencies" : ["USD"]}

payload = {}

for index, segment in enumerate(data["segments"]):
    origin = segment["origin"]
    destination = segment["destination"]
    # python 3.6+ needed:
    payload[f"segments[{index}][origin]"] = origin
    payload[f"segments[{index}][destination]"] = destination

payload["cabin_class"] = data["cabin_class"]

# requests can handle repeated parameters with the same name this way:
payload["currencies[]"] = data["currencies"]
print(payload)

response = requests.get(
    "https://api.goclimate.com/v1/flight_footprint",
    auth=("464c01f7e53b2a5e6f00d9a1", ""),
    params=payload, 
)

def addObject(file):
    print("API")
    print(file.content)
    d = response.text
    convertedDict = json.loads(d)
    print(convertedDict["footprint"])
    #print(convertedDict["offset_prices"]["amount"])

    testCSV["Footprint"] = convertedDict["footprint"]
    # testCSV["Price USD"] = convertedDict["offset_prices"]["amount"]
    print(testCSV)
addObject(response)

{'segments[0][origin]': 'ZRH', 'segments[0][destination]': 'ATL', 'cabin_class': 'economy', 'currencies[]': ['USD']}
API
b'{"footprint":1500,"offset_prices":[{"amount":2550,"currency":"USD","offset_url":"https://www.goclimate.com/us/flight_offsets/new?offset_params=economy%2CZRH%2CATL","locale":"en-US"}],"details_url":"https://www.goclimate.com/se/flight_offsets/new?offset_params=economy%2CZRH%2CATL"}'
1500
       Unnamed: 0   RecID     year Flugnummer cabin_class  DEPARTURE_AIRPORT  \
1  1         2020  LX4425  economy        CGN         ZRH                  1   
2  2         2020   OS420  economy        CDG         VIE                  1   
3  3         2020   OS553  economy        VIE         ZRH                  1   
4  4         2020   OS763  economy        VIE         VAR                  1   
5  5         2020   OS764  economy        VAR         VIE                  1   
6  6         2020  SU2392  economy        NaN         NaN                  1   
7  7         2020  SU2393  ec